# Canada Wildfires Analysis - STAT 201 Project Proposal

## 1. Introduction

Text here.

### Research Question and Parameter

### Identify and Describe the Dataset

## 2. Preliminary Results

### Load libraries

In [10]:
### Run this cell before continuing.
library(cowplot)
library(dplyr)
library(gridExtra)
library(tidyverse)
library(repr)
library(infer)
library(repr)

### Read dataset from web into R

In [9]:
url <- 'https://gist.githubusercontent.com/hd54/d45ccf80e72b9c87dbc636fb9d33af93/raw/ec514840169031c4eff262d5c42474a44c8d728f/gistfile1.txt'
wildfires <- read_tsv(url)

head(wildfires)

Rows: 423831 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (5): SRC_AGENCY, REP_DATE, CAUSE, PROTZONE, ECOZ_NAME
dbl (4): FID, LATITUDE, LONGITUDE, SIZE_HA

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


FID,SRC_AGENCY,LATITUDE,LONGITUDE,REP_DATE,SIZE_HA,CAUSE,PROTZONE,ECOZ_NAME
<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>
0,BC,59.963,-128.172,5/26/1953,8.0,H,NA,Boreal Cordillera
1,BC,59.318,-132.172,6/22/1950,8.0,L,NA,Boreal Cordillera
2,BC,59.876,-131.922,6/4/1950,12949.9,H,NA,Boreal Cordillera
3,BC,59.760,-132.808,7/15/1951,241.1,H,NA,Boreal Cordillera
4,BC,59.434,-126.172,6/12/1952,1.2,H,NA,Boreal Cordillera
5,BC,59.963,-136.502,8/1/1951,194.2,H,NA,Boreal Cordillera


In [11]:
# wrangling data

In [12]:
# visualize data

### 3. Methods: Plan

### Report Summary
Why is this report trustworthy?

### Hypothesis Test:
- **Null Hypothesis $H_{0}$**:
- **Alternative Hypothesis $H_{\alpha}$**:
- **Significance Level (α):** 

### Confidence Intervals:
- **Parameters of Interest**:
- **Confidence Level:**

What do you expect to find?

What impact could such findings have?

What future questions could this lead to?

### 4. References